In [137]:
from sklearn.linear_model import LogisticRegression
from timeit import default_timer as timer
from datetime import timedelta
import os
from sklearn import preprocessing
import numpy as np
from read_scripts import dict_2_arr 
from read_scripts import read_dataset 
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor

task = "remote_homology"

def fit_logistic(X, y):
    start = timer()
    clf = LogisticRegression(max_iter=5000)
    clf.fit(X, y)
    end = timer()
    print(f"fit time: ", timedelta(seconds=end-start))
    
    train_score = clf.score(X, y)
    print(f"model train score: ", train_score)
    
    # when run will play a ping sound!
    os.system("printf '\a'")
    os.system("printf '\a'")
    os.system("printf '\a'")
    print("Done!")
    
    return clf

In [138]:
def get_single_model_normalized_corpus(model):
    ys = []
    Xs = []

    # read all splits
    for split in ['train', 'valid', 'test_fold_holdout', 'test_superfamily_holdout', 'test_family_holdout']:

        y_dict = read_dataset('label', task, split)
        X, y = dict_2_arr(read_dataset(model, task, split), y_dict)
        
        ys.append(y)
        Xs.append(X)  
    
    # concat all splits
    corpus = np.concatenate(Xs, axis=0)
    
    return corpus, ys

In [172]:
def get_combined_normalized_corpus_weighted(weights):
    ys = []
    els = []
    trfs = []
    unis = []

    # read all splits
    for split in ['train', 'valid', 'test_fold_holdout', 'test_superfamily_holdout', 'test_family_holdout']:

        y_dict = read_dataset('label', task, split)
        X_e, y = dict_2_arr(read_dataset('elmo', task, split), y_dict)
        X_t, y = dict_2_arr(read_dataset('transformer', task, split), y_dict)
        X_u, y = dict_2_arr(read_dataset('unirep', task, split), y_dict)

        ys.append(y)
        els.append(X_e)  
        trfs.append(X_t)
        unis.append(X_u)
    
    # concat all splits
    e_corpus = np.concatenate(els, axis=0)
    t_corpus = np.concatenate(trfs, axis=0)
    u_corpus = np.concatenate(unis, axis=0)
    
    # normalize each indvidually 
    e_corpus = preprocessing.normalize(e_corpus, norm='l2') * weights[0]
    t_corpus = preprocessing.normalize(t_corpus, norm='l2') * weights[1]
    u_corpus = preprocessing.normalize(u_corpus, norm='l2') * weights[2]
    
    # concatenate all corpuses
    combined = np.concatenate([e_corpus, t_corpus, u_corpus], axis=1)
    
    return combined, ys

In [173]:
def train_test_over_corpus(corpus, ys, model_name):

    all_score = {}

    # get out training slice
    X_train = corpus[:len(ys[0])]
    y_train = ys[0]

    # scale
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)

    # fit!
    clf = fit_logistic(X_train, y_train)

    # record train score
    train_score = clf.score(X_train, y_train)
    print(f"{model_name} train score: ", train_score)

    all_score["train"]  = train_score

    # get slices for remaing splits and score
    remain_splits = ['valid', 'test_fold_holdout', 'test_superfamily_holdout', 'test_family_holdout']
    end = len(ys[0])
    for i in range(len(remain_splits)):
        split = remain_splits[i]
        start = end
        end = len(ys[i+1])+start

        X = corpus[start:end]
        y = ys[i+1]

        X = scaler.transform(X)

        test_score = clf.score(X, y)

        all_score[split]  = test_score

        print(f"{model_name} {split} score: ", test_score)

    return all_score

In [174]:
def train_auto_encoder(X, latent_dim, hidden_layers):

    n_input = X.shape[1]

    full_layers = hidden_layers + [latent_dim] + hidden_layers[::-1]
    print("layers", full_layers)
    
    reg = MLPRegressor(hidden_layer_sizes = full_layers, 
                       activation = 'relu', 
                       solver = 'adam', 
                       learning_rate_init = 0.0001, 
                       max_iter = 100, 
                       tol = 0.0000001, 
                       verbose = False)
    
    print("fitting auto_encoder...")
    reg.fit(X, X)
    
    auto_encoder_train_score = reg.score(X, X)
    print("auto_encoder_train_score: ", auto_encoder_train_score)
    return reg


def encoder(X, reg):
    print("encoding...")
    X = np.asmatrix(X)
    
    encoder1 = X*reg.coefs_[0] + reg.intercepts_[0]
    encoder1 = (np.exp(encoder1) - np.exp(-encoder1))/(np.exp(encoder1) + np.exp(-encoder1))
    
    encoder2 = encoder1*reg.coefs_[1] + reg.intercepts_[1]
    encoder2 = (np.exp(encoder2) - np.exp(-encoder2))/(np.exp(encoder2) + np.exp(-encoder2))
    
    latent = encoder2*reg.coefs_[2] + reg.intercepts_[2]
    latent = (np.exp(latent) - np.exp(-latent))/(np.exp(latent) + np.exp(-latent))
    
    print("encoded as: ", latent.shape)
    
    return np.asarray(latent)

def train_and_encode(X, latent_dim=200, hidden_layers=[500, 300]):
    
    reg = train_auto_encoder(X, latent_dim, hidden_layers)
    X_encoded = encoder(X, reg)
    
    return X_encoded

In [175]:
def do_conact_then_auto_encode_for_dim(X, ys, dim):
    
    hidden_layers = [1200, int((d+1200)/2)]
    X = train_and_encode(X, latent_dim=dim, hidden_layers=hidden_layers)
    scores = train_test_over_corpus(X, ys, f"concat_then_auto_encode dim {dim}")
    return scores

In [176]:
X, ys = get_combined_normalized_corpus_weighted([6,3,1])

In [177]:
dims = []
for d in [10, 20, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700]:
    dims.append(d)
all_dim_all_scores = {}

for d in dims:
    s = do_conact_then_auto_encode_for_dim(X, ys, d)
    all_dim_all_scores[str(d)] = s

layers [1200, 605, 10, 605, 1200]
fitting auto_encoder...


/Users/julesberman/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


auto_encoder_train_score:  -0.18793691247078564
encoding...
encoded as:  (16281, 10)
fit time:  0:02:03.444124
model train score:  0.38228362454286874
Done!
concat_then_auto_encode dim 10 train score:  0.38228362454286874
concat_then_auto_encode dim 10 valid score:  0.14032697547683923
concat_then_auto_encode dim 10 test_fold_holdout score:  0.13370473537604458
concat_then_auto_encode dim 10 test_superfamily_holdout score:  0.17252396166134185
concat_then_auto_encode dim 10 test_family_holdout score:  0.5487421383647799
layers [1200, 610, 20, 610, 1200]
fitting auto_encoder...


/Users/julesberman/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


auto_encoder_train_score:  -0.2058248321870798
encoding...
encoded as:  (16281, 20)
fit time:  0:02:19.995402
model train score:  0.6007314099959367
Done!
concat_then_auto_encode dim 20 train score:  0.6007314099959367
concat_then_auto_encode dim 20 valid score:  0.21934604904632152
concat_then_auto_encode dim 20 test_fold_holdout score:  0.18802228412256267
concat_then_auto_encode dim 20 test_superfamily_holdout score:  0.2715654952076677
concat_then_auto_encode dim 20 test_family_holdout score:  0.7287735849056604
layers [1200, 625, 50, 625, 1200]
fitting auto_encoder...


...

auto_encoder_train_score:  -0.1770244400487152
encoding...
encoded as:  (16281, 600)
fit time:  0:09:42.607633
model train score:  1.0
Done!
concat_then_auto_encode dim 600 train score:  1.0
concat_then_auto_encode dim 600 valid score:  0.3923705722070845
concat_then_auto_encode dim 600 test_fold_holdout score:  0.24512534818941503
concat_then_auto_encode dim 600 test_superfamily_holdout score:  0.44568690095846647
concat_then_auto_encode dim 600 test_family_holdout score:  0.9363207547169812
layers [1200, 925, 650, 925, 1200]
fitting auto_encoder...


/Users/julesberman/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


auto_encoder_train_score:  -0.3892306316814977
encoding...
encoded as:  (16281, 650)
fit time:  0:10:16.235997
model train score:  1.0
Done!
concat_then_auto_encode dim 650 train score:  1.0
concat_then_auto_encode dim 650 valid score:  0.38419618528610355
concat_then_auto_encode dim 650 test_fold_holdout score:  0.2562674094707521
concat_then_auto_encode dim 650 test_superfamily_holdout score:  0.4520766773162939
concat_then_auto_encode dim 650 test_family_holdout score:  0.9339622641509434
layers [1200, 950, 700, 950, 1200]
fitting auto_encoder...


/Users/julesberman/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


auto_encoder_train_score:  -0.14419528035556914
encoding...
encoded as:  (16281, 700)
fit time:  0:09:48.001611
model train score:  1.0
Done!
concat_then_auto_encode dim 700 train score:  1.0
concat_then_auto_encode dim 700 valid score:  0.3814713896457766
concat_then_auto_encode dim 700 test_fold_holdout score:  0.2479108635097493
concat_then_auto_encode dim 700 test_superfamily_holdout score:  0.4496805111821086
concat_then_auto_encode dim 700 test_family_holdout score:  0.9378930817610063


In [178]:
import pickle
with open("concat_then_auto_encode_across_dims_weighted_631.p", "wb") as f:
    pickle.dump(all_dim_all_scores, f)